# Transaction proof
This part of the code will cary out the verification that the job is been done and sent to the client. 
It will be organized into three separate parts:
- First part: A proof-based reasoning behind the object having been sent. More math-based.
- Second part: Allows the client to verify their satisfaction with the product received. Prevents ghosting.
- Third part: Allows the freelancer to receive the payment.


## First part - Mathematical reasoning behind the transaction being done
This code is mathematical and deterministic, but doesn not offer any way to validate the quality of the object that is sent.

The code does the following things:
- Makes sure that the data has been sent

- Prevents the data from being modified once sent

Flare will be used. 

Everything is localised in Python, so we can't achieve the trustless model we are looking for and the code relies on the Python database. Flare allows for a globalization of certain parameters like timestamps, order of transactions and hashes.

In [22]:
!pip3 install web3 #To make sure it is downloaded inside the Kernel
 



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip


In [23]:
import time
import hashlib #For the cryptography functions
from enum import Enum

In [24]:
from web3 import Web3

# Conectar a Flare Testnet (Coston2)
flare_rpc = "https://coston2-api.flare.network/ext/C/rpc"
w3 = Web3(Web3.HTTPProvider(flare_rpc))

# Verificar conexión
print("Connected:", w3.is_connected())

# Leer número de bloque actual
print("Current block:", w3.eth.block_number)


Connected: True
Current block: 26981684


In [25]:
class FlareProofOfDelivery:
    def __init__(self, freelancer_id: str, data: str):
        self.freelancer_id = freelancer_id #Id of freelancer
        self.timestamp = time.time()
        self.hash = generate_hash(data) #Proof the the data exists and if the data is modified
                                        #the hash will be modified
        self.block_number = None  # It will link itself to Flare, so far there is no block number

    def anchor_on_flare(self):
        # The transaction is published on Flare
        tx = w3.eth.send_transaction({
            'from': w3.eth.accounts[0],  
            'to': w3.eth.accounts[0],    
            'value': 0,
            'data': self.hash.encode()
         }) #Hash is pblished and uploaded on all Flare nodes, unmutable
        receipt = w3.eth.wait_for_transaction_receipt(tx)
        self.block_number = receipt.blockNumber #Stores number of block where transaction was produced
        #verifies when the transaction took place and is universal for all.
        print(f"Proof anchored on Flare at block {self.block_number}") #Confirmation



## Second part - Is it up to expectations?
This step will now verify that the client is satisfied with the product received and it matches their expectations.

The code does the following things:
- For the payment to be carried out, the freelancer must have sent the data
- There are no deadlocks
- Prevents ghosting

Flare allows for:
- A globalised enviroment, safe from manipulation
- Prevents any manipulation time-wise


In [26]:
class FlareVerifier:
    def __init__(self, proof: FlareProofOfDelivery, expected_hash: str):
        self.proof = proof #Stores the hash
        self.expected_hash = expected_hash
        self.result = VerificationResult.PENDING

    def verify(self):
        # Verifica hash y existencia en blockchain
        if self.proof.hash != self.expected_hash:
            self.result = VerificationResult.REJECTED
        elif self.proof.block_number is None:
            self.result = VerificationResult.DISPUTED
        else:
            self.result = VerificationResult.ACCEPTED #Whole process of the client acceopting the product
        print(f"Verification result: {self.result.name}")
        return self.result #Verification



## Third part - Dictates what occurs with the money
Once the data has been sent and accepted by the consumer, then the money, uploaded on Lancelot itself, is ent over to the user.

This code does the following things:
- Transmits the payment only once the freelancer has sent the product and the client has accepted it

The use of Flare allows:
- Payment is only carried out if Flare approves it
- The sequence is unmodifible and easily verified by the freelancer/client

In [27]:
class FlareSettlementEngine:
    def __init__(self, client_balance: int, freelancer_balance: int, payment: int):
        self.client_balance = client_balance
        self.freelancer_balance = freelancer_balance
        self.payment = payment #Defining the balances and payment

    def settle(self, verification: FlareVerifier): #process of settling the pay
        if verification.result == VerificationResult.ACCEPTED: 
            self.client_balance -= self.payment
            self.freelancer_balance += self.payment
            print(f"Payment of {self.payment} settled. Freelancer new balance: {self.freelancer_balance}")
        elif verification.result == VerificationResult.REJECTED:
            print("Verification failed. Payment not released.")
        elif verification.result == VerificationResult.DISPUTED:
            print("Verification disputed. Payment frozen until resolution.")


## Sample
In this case, we will define a freelancer and client, each with 1000 units in their account and tne transaction is 200 units. The product desired is an empty list.

We will try the code for an empty and non-empty list.

In [28]:
#Conection to Flare
flare_rpc = "https://coston2-api.flare.network/ext/C/rpc"
w3 = Web3(Web3.HTTPProvider(flare_rpc))

if not w3.is_connected():
    raise Exception("Not able to connect to Flare")
print("Connected to Flare! Block:", w3.eth.block_number)

# Functions and enumerations
class VerificationResult(Enum):
    PENDING = 0
    ACCEPTED = 1
    REJECTED = 2
    DISPUTED = 3

def generate_hash(data) -> str:
    # Convertir cualquier dato a string para generar hash
    return hashlib.sha256(str(data).encode()).hexdigest()

#Part 1
class FlareProofOfDelivery:
    def __init__(self, freelancer_id, data):
        self.freelancer_id = freelancer_id
        self.timestamp = time.time()
        self.hash = generate_hash(data)
        self.block_number = None

    def anchor_on_flare(self):
        # Simulación: asignamos el bloque actual
        self.block_number = w3.eth.block_number
        print(f"Proof anchored on Flare at block {self.block_number}")

#Part 2
class FlareVerifier:
    def __init__(self, proof, expected_hash):
        self.proof = proof
        self.expected_hash = expected_hash
        self.result = VerificationResult.PENDING

    def verify(self):
        if self.proof.hash != self.expected_hash:
            self.result = VerificationResult.REJECTED
        elif self.proof.block_number is None:
            self.result = VerificationResult.DISPUTED
        else:
            self.result = VerificationResult.ACCEPTED
        print(f"Verification result: {self.result.name}")
        return self.result

#Part 3
class FlareSettlementEngine:
    def __init__(self, client_balance, freelancer_balance, payment):
        self.client_balance = client_balance
        self.freelancer_balance = freelancer_balance
        self.payment = payment

    def settle(self, verification):
        if verification.result == VerificationResult.ACCEPTED:
            self.client_balance -= self.payment
            self.freelancer_balance += self.payment
            print(f"Payment of {self.payment} settled. Freelancer new balance: {self.freelancer_balance}")
        elif verification.result == VerificationResult.REJECTED:
            print("Verification failed. Payment not released.")
        elif verification.result == VerificationResult.DISPUTED:
            print("Verification disputed. Payment frozen until resolution.")

#Function to prove products
def test_delivery(freelancer_product, expected_product):
    print("\n=== Testing delivery ===")
    client_balance = 1000
    freelancer_balance = 1000
    payment = 200

    # Part 1
    proof = FlareProofOfDelivery("freelancer_1", freelancer_product)
    proof.anchor_on_flare()

    # Part 2
    expected_hash = generate_hash(expected_product)
    verifier = FlareVerifier(proof, expected_hash)
    verifier.verify()

    # Part 3
    settlement = FlareSettlementEngine(client_balance, freelancer_balance, payment)
    settlement.settle(verifier)

    print("Final balances: Client =", settlement.client_balance, "Freelancer =", settlement.freelancer_balance)

expected_product = []

# Case 1 - Accepted
test_delivery([], expected_product)

# Case 2 - Rejected
test_delivery(["item1"], expected_product)

Connected to Flare! Block: 26981684

=== Testing delivery ===
Proof anchored on Flare at block 26981684
Verification result: ACCEPTED
Payment of 200 settled. Freelancer new balance: 1200
Final balances: Client = 800 Freelancer = 1200

=== Testing delivery ===
Proof anchored on Flare at block 26981684
Verification result: REJECTED
Verification failed. Payment not released.
Final balances: Client = 1000 Freelancer = 1000


Let's now change it so that the product and expected product depend on input functions, as well as the cost.

In [29]:


#Conection to Flare
flare_rpc = "https://coston2-api.flare.network/ext/C/rpc"
w3 = Web3(Web3.HTTPProvider(flare_rpc))

if not w3.is_connected():
    raise Exception("Not able to connect to Flare")
print("Connected to Flare! Block:", w3.eth.block_number)

# Functions and enumerations
class VerificationResult(Enum):
    PENDING = 0
    ACCEPTED = 1
    REJECTED = 2
    DISPUTED = 3

def generate_hash(data) -> str:
    # Convertir cualquier dato a string para generar hash
    return hashlib.sha256(str(data).encode()).hexdigest()

#Part 1
class FlareProofOfDelivery:
    def __init__(self, freelancer_id, data):
        self.freelancer_id = freelancer_id
        self.timestamp = time.time()
        self.hash = generate_hash(data)
        self.block_number = None

    def anchor_on_flare(self):
        # Simulación: asignamos el bloque actual
        self.block_number = w3.eth.block_number
        print(f"Proof anchored on Flare at block {self.block_number}")

#Part 2
class FlareVerifier:
    def __init__(self, proof, expected_hash):
        self.proof = proof
        self.expected_hash = expected_hash
        self.result = VerificationResult.PENDING

    def verify(self):
        if self.proof.hash != self.expected_hash:
            self.result = VerificationResult.REJECTED
        elif self.proof.block_number is None:
            self.result = VerificationResult.DISPUTED
        else:
            self.result = VerificationResult.ACCEPTED
        print(f"Verification result: {self.result.name}")
        return self.result

#Part 3
class FlareSettlementEngine:
    def __init__(self, client_balance, freelancer_balance, payment):
        self.client_balance = client_balance
        self.freelancer_balance = freelancer_balance
        self.payment = payment

    def settle(self, verification):
        if verification.result == VerificationResult.ACCEPTED:
            self.client_balance -= self.payment
            self.freelancer_balance += self.payment
            print(f"Payment of {self.payment} settled. Freelancer new balance: {self.freelancer_balance}")
        elif verification.result == VerificationResult.REJECTED:
            print("Verification failed. Payment not released.")
        elif verification.result == VerificationResult.DISPUTED:
            print("Verification disputed. Payment frozen until resolution.")

#Function to prove products
def test_delivery(freelancer_product, expected_product):
    print("\n=== Testing delivery ===")
    client_balance = 1000
    freelancer_balance = 1000
    payment = float(input("Cost of the product"))

    # Part 1
    proof = FlareProofOfDelivery("freelancer_1", freelancer_product)
    proof.anchor_on_flare()

    # Part 2
    expected_hash = generate_hash(expected_product)
    verifier = FlareVerifier(proof, expected_hash)
    verifier.verify()

    # Part 3
    settlement = FlareSettlementEngine(client_balance, freelancer_balance, payment)
    settlement.settle(verifier)

    print("Final balances: Client =", settlement.client_balance, "Freelancer =", settlement.freelancer_balance)

expected_product = input("What is the expected product?")

product = input("What is the product?")
test_delivery(product, expected_product)



Connected to Flare! Block: 26981684

=== Testing delivery ===
Proof anchored on Flare at block 26981686
Verification result: ACCEPTED
Payment of 9.0 settled. Freelancer new balance: 1009.0
Final balances: Client = 991.0 Freelancer = 1009.0
